# User Notebook

In [9]:
import pandas as pd
import numpy as np
import pyodbc
from data_prep_functions import *

In [139]:
households_csv = download_and_concat_Tdrive_files('42', 'Household_Files')

In [140]:
households_csv

,hhid,household_serial_no,taz,mgra,hinccat1,hinc,hworkers,veh,persons,hht,bldgsz,unittype,version,poverty,year
0,1,0,3458,47,5,294757,1,0,2,1,2,0,0,22.364,2016
1,2,0,3407,10,5,210480,1,0,2,1,2,0,0,15.970,2016
2,3,0,3458,16,5,236389,0,0,2,3,2,0,0,15.728,2016
3,4,0,3379,27,4,121977,2,0,2,2,2,0,0,8.353,2016
4,5,0,3358,3,4,104512,2,0,2,7,2,0,0,7.157,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455750,1455751,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455751,1455752,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455752,1455753,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455753,1455754,0,3144,6055,1,14000,1,0,1,0,9,1,0,1.234,2050


In [ ]:
households_csv = households_csv[households_csv['unittype'] != 1]

In [10]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=isam;'
                      'Trusted_Connection=yes;')

In [11]:
query_all = """SELECT hhid, household_serial_no, taz, mgra, hinccat1, hinc, hworkers, veh, persons, hht, bldgsz, unittype, version, poverty, yr AS 'year'  
FROM ddamwsql16.isam.xpef36.abm_syn_households
WHERE yr in (2016, 2018, 2020, 2023, 2025, 2026, 2029, 2030, 2032, 2035, 2040, 2045, 2050)"""

In [12]:
sql_query = pd.read_sql_query(query_all,conn)
households_sql = pd.DataFrame(sql_query)

In [23]:
households_sql['year'] = households_sql['year'].astype('str')

In [24]:
list(set(households_sql['year']))

['2030', '2018', '2026']

In [26]:
households_csv_cleaned = households_csv[households_csv['year'].isin(list(set(households_sql['year'])))]

In [47]:
households_csv_cleaned.to_csv(r'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Service Requests/2022/2022-81 XPEF persons vs mgra13_based_inputs/Data/households_from_csv_filtered.csv')

In [48]:
households_sql.to_csv(r'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Service Requests/2022/2022-81 XPEF persons vs mgra13_based_inputs/Data/households_from_sql.csv')

In [27]:
# Checking Shape
print(households_csv_cleaned.shape)
print(households_sql.shape)

(3882908, 15)
(3846960, 15)


In [37]:
3882908-3846960

35948

In [39]:
(1355447-1344364) * 3

33249

In [30]:
print(set(households_csv_cleaned['year']))
print(set(households_sql['year']))

{'2030', '2018', '2026'}
{'2030', '2018', '2026'}


In [31]:
print(households_csv_cleaned.columns == households_sql.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True]


In [36]:
print(households_csv_cleaned['hhid'].max())
print(households_sql['hhid'].max())

1355447
1344364


In [34]:
households_sql.sort_values('hhid')

,hhid,household_serial_no,taz,mgra,hinccat1,hinc,hworkers,veh,persons,hht,bldgsz,unittype,version,poverty,year
898616,1,0,3458,47,4,140111,0,0,2,1,2,0,0,10.631,2026
5544,1,0,3458,47,5,275519,1,0,2,1,2,0,0,20.904,2018
1640895,1,0,3358,3,5,160615,1,0,2,1,2,0,0,12.186,2030
5545,2,0,3407,10,3,98703,1,0,2,1,2,0,0,7.489,2018
1640896,2,0,3407,7,1,9339,0,0,1,6,2,0,0,0.893,2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3234761,1344360,0,1013,19191,1,14000,0,0,1,0,9,1,0,1.234,2030
3234762,1344361,0,1013,19191,1,14000,1,0,1,0,9,1,0,1.234,2030
3234763,1344362,0,1013,19191,1,14000,0,0,1,0,9,1,0,1.234,2030
3234764,1344363,0,1013,19191,1,14000,1,0,1,0,9,1,0,1.234,2030


In [41]:
in_csv_only = households_csv_cleaned.merge(households_sql, on=list(households_sql.columns), how='left', indicator=True)
in_csv_only

,hhid,household_serial_no,taz,mgra,hinccat1,hinc,hworkers,veh,persons,hht,bldgsz,unittype,version,poverty,year,_merge
0,1,0,3458,47,1,28845,0,0,2,1,2,0,0,2.189,2018,left_only
1,2,0,3407,10,5,285674,0,0,2,1,2,0,0,21.675,2018,left_only
2,3,0,3458,16,3,97220,0,0,2,3,2,0,0,6.468,2018,left_only
3,4,0,3379,27,2,33960,1,0,2,2,2,0,0,2.326,2018,left_only
4,5,0,3358,3,5,232448,2,0,2,7,2,0,0,15.919,2018,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3882903,1355443,0,1013,19191,1,14000,0,0,1,0,9,1,0,1.234,2030,left_only
3882904,1355444,0,1013,19191,1,14000,0,0,1,0,9,1,0,1.234,2030,left_only
3882905,1355445,0,1013,19191,1,14000,0,0,1,0,9,1,0,1.234,2030,left_only
3882906,1355446,0,1013,19191,1,14000,1,0,1,0,9,1,0,1.234,2030,left_only


In [43]:
in_csv_only[in_csv_only['_merge'] == 'left_only']

,hhid,household_serial_no,taz,mgra,hinccat1,hinc,hworkers,veh,persons,hht,bldgsz,unittype,version,poverty,year,_merge
0,1,0,3458,47,1,28845,0,0,2,1,2,0,0,2.189,2018,left_only
1,2,0,3407,10,5,285674,0,0,2,1,2,0,0,21.675,2018,left_only
2,3,0,3458,16,3,97220,0,0,2,3,2,0,0,6.468,2018,left_only
3,4,0,3379,27,2,33960,1,0,2,2,2,0,0,2.326,2018,left_only
4,5,0,3358,3,5,232448,2,0,2,7,2,0,0,15.919,2018,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3882903,1355443,0,1013,19191,1,14000,0,0,1,0,9,1,0,1.234,2030,left_only
3882904,1355444,0,1013,19191,1,14000,0,0,1,0,9,1,0,1.234,2030,left_only
3882905,1355445,0,1013,19191,1,14000,0,0,1,0,9,1,0,1.234,2030,left_only
3882906,1355446,0,1013,19191,1,14000,1,0,1,0,9,1,0,1.234,2030,left_only


In [46]:
in_sql_only = households_sql.merge(households_csv_cleaned, on=list(households_sql.columns), how='left', indicator=True)
in_sql_only[in_sql_only['_merge'] == 'left_only']

,hhid,household_serial_no,taz,mgra,hinccat1,hinc,hworkers,veh,persons,hht,bldgsz,unittype,version,poverty,year,_merge
0,18382,0,3655,783,1,25809,2,0,2,1,2,0,0,1.767,2018,left_only
1,18383,0,3655,787,1,21326,1,0,3,3,3,0,0,1.214,2018,left_only
2,18384,0,3595,806,4,142002,2,0,2,1,8,0,0,9.725,2018,left_only
3,18385,0,3595,796,3,87979,3,0,4,1,2,0,0,3.912,2018,left_only
4,18386,0,3655,785,1,3782,2,0,8,1,2,0,0,0.099,2018,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3846955,735284,0,2202,4669,1,5321,0,0,4,1,8,0,0,0.241,2018,left_only
3846956,735285,0,3391,8888,3,78863,2,0,5,1,2,0,0,3.031,2018,left_only
3846957,735286,0,3645,9425,1,9497,1,0,4,3,8,0,0,0.429,2018,left_only
3846958,735287,0,3663,8784,4,128296,1,0,4,3,2,0,0,5.802,2018,left_only


In [49]:
import yaml

In [ ]:
# Persons and Houshold Commparison 

In [169]:
def download_individual_persons_file(dsid, year):
    """Downloads an individual person file for one particular year. Year is a string value."""

    with open('ds_config_2.yml', "r") as yml_file:
        config = yaml.safe_load(yml_file)

    file_path =  config[dsid]['Person_Files'][f'DS{dsid}_persons_{year}']
    return pd.read_csv(file_path, usecols=['hhid', 'miltary'])

In [170]:
def download_individual_households_file_for_person_comp(dsid, year):
    """Downloads an individual person file for one particular year. Year is a string value."""

    with open('ds_config_2.yml', "r") as yml_file:
        config = yaml.safe_load(yml_file)

    file_path =  config[dsid]['Household_Files'][f'DS{dsid}_households_{year}']
    return pd.read_csv(file_path, usecols=['hhid', 'persons', 'unittype'])

In [171]:
def persons_dataset_hhid_population(dsid, year, gq_only):
    persons_df = download_individual_persons_file(dsid, year)

    if gq_only:
        persons_df = persons_df[persons_df['miltary'] == 1]
    
    persons_df = persons_df.groupby('hhid').count()
    persons_df.columns = ['Persons_Dataset_Pop']
    
    return persons_df.reset_index()


In [172]:
def persons_households_dataset_pop_comparison(dsid, year, gq_only):
    """This functions joins the persons and household datatogether and """
    persons_df = persons_dataset_hhid_population(dsid, year, gq_only)
    households_df = download_individual_households_file_for_person_comp(dsid, year)

    if gq_only: # This gets handled for persons in the "persons_dataset_hhid_population" function
        households_df = households_df[households_df['unittype'] == 1]
    
    # Rename Household df
    households_df = households_df.rename(columns={'persons':'Households_Dataset_Pop'})

    # Grab columns of interest 
    persons_df = persons_df[['hhid', 'Persons_Dataset_Pop']]
    households_df = households_df[['hhid', 'Households_Dataset_Pop']]
    
    # Combine
    output = persons_df.merge(households_df, how='left', on='hhid')

    # Diff
    output['Diff_P_minus_H'] =  output['Persons_Dataset_Pop'] - output['Households_Dataset_Pop']

    # Add year
    output['year'] = year

    return output


In [173]:
def find_individual_years_for_dsid(dsid):
    "Takes in a dsid and returns a list of years that this dsid covers."
    with open('ds_config_2.yml', "r") as yml_file:
        config = yaml.safe_load(yml_file)

    t_drive_file_keys = config[dsid]['T_Drive_files'].keys()

    return [x[-4:] for x in t_drive_file_keys]
    

In [176]:
def aggregate_persons_households_population_comparison(dsid, gq_only, to_jdrive):
    concatonated_dfs = pd.DataFrame()

    for year in find_individual_years_for_dsid(dsid):
        temp_df = persons_households_dataset_pop_comparison(dsid, year, gq_only)
        concatonated_dfs = pd.concat([concatonated_dfs, temp_df])
        del temp_df
        print(f"{year} is complete")

    if gq_only:
        gq_status = 'gq_only'
    else:
        gq_status = 'all'

    if to_jdrive: 
        concatonated_dfs.to_csv(
            rf"J:\DataScience\DataQuality\QAQC\forecast_automation\mgra_series_13_outputs_CSV_data\other_outputs\DS{dsid}_persons_household_population_comparison_{gq_status}_QA.csv", index=False)

    return concatonated_dfs

In [177]:
d = aggregate_persons_households_population_comparison('42', gq_only=False, to_jdrive=True)
d

2016 is complete
2018 is complete
2020 is complete
2023 is complete
2025 is complete
2026 is complete
2029 is complete
2030 is complete
2032 is complete
2035 is complete
2040 is complete
2045 is complete
2050 is complete


,hhid,Persons_Dataset_Pop,Households_Dataset_Pop,Diff_P_minus_H,year
0,1,2,2,0,2016
1,2,2,2,0,2016
2,3,2,2,0,2016
3,4,2,2,0,2016
4,5,2,2,0,2016
...,...,...,...,...,...
1455750,1455751,1,1,0,2050
1455751,1455752,1,1,0,2050
1455752,1455753,1,1,0,2050
1455753,1455754,1,1,0,2050


In [179]:
d[d['Diff_P_minus_H'] != 0]

,hhid,Persons_Dataset_Pop,Households_Dataset_Pop,Diff_P_minus_H,year


In [178]:
e = aggregate_persons_households_population_comparison('42', gq_only=True, to_jdrive=True)
e

2016 is complete
2018 is complete
2020 is complete
2023 is complete
2025 is complete
2026 is complete
2029 is complete
2030 is complete
2032 is complete
2035 is complete
2040 is complete
2045 is complete
2050 is complete


,hhid,Persons_Dataset_Pop,Households_Dataset_Pop,Diff_P_minus_H,year
0,886,1,NaN,NaN,2016
1,2000,1,NaN,NaN,2016
2,2554,1,NaN,NaN,2016
3,2608,1,NaN,NaN,2016
4,2852,1,NaN,NaN,2016
...,...,...,...,...,...
84871,1453334,1,1.0,0.0,2050
84872,1453335,1,1.0,0.0,2050
84873,1453336,1,1.0,0.0,2050
84874,1453337,1,1.0,0.0,2050


In [180]:
e[e['Diff_P_minus_H'] != 0]

,hhid,Persons_Dataset_Pop,Households_Dataset_Pop,Diff_P_minus_H,year
0,886,1,NaN,NaN,2016
1,2000,1,NaN,NaN,2016
2,2554,1,NaN,NaN,2016
3,2608,1,NaN,NaN,2016
4,2852,1,NaN,NaN,2016
...,...,...,...,...,...
33544,1373120,1,NaN,NaN,2050
33545,1373472,1,NaN,NaN,2050
33546,1373988,1,NaN,NaN,2050
33547,1374010,1,NaN,NaN,2050


In [146]:
c = persons_households_dataset_pop_comparison('42', '2016', gq_only=True)
c

KeyError: 'miltary'

In [106]:
persons_dataset_hhid_population('42', '2016')

,hhid,Persons_Dataset_Pop
0,1,2
1,2,2
2,3,2
3,4,2
4,5,2
5,6,5
6,7,2
7,8,2
8,9,4
9,10,4


In [77]:
a = download_persons_file('42', '2016')
a

,hhid,miltary
0,1,0
1,1,0
2,2,0
3,2,0
4,3,0
...,...,...
95,37,0
96,38,0
97,38,0
98,38,0


In [2]:
output = population_comparison_households_and_input_files('42', gq_only=False, no_gq=True, to_jdrive=True)
output

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,pop_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
236889,2050,22995,99,99,0
236890,2050,22996,269,269,0
236891,2050,22998,221,221,0
236892,2050,23000,282,282,0


In [3]:
output[output['Diff'] != 0]

,year,mgra,pop_count_household_file,pop_input_files,Diff


In [3]:
ds42_households = download_and_concat_Tdrive_files('42', 'Household_Files')

In [4]:
# ds42_households.to_csv(r'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Service Requests/2022/2022-81 XPEF persons vs mgra13_based_inputs/Data/ds42_households.csv')

In [5]:
ds42_persons = download_and_concat_Tdrive_files('42', 'Person_Files')

In [ ]:
# ds42_persons.to_csv(r'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Service Requests/2022/2022-81 XPEF persons vs mgra13_based_inputs/Data/ds42_persons.csv')

In [3]:
t_drive_42_data = download_and_concat_Tdrive_files('42', 'T_Drive_files')
t_drive_99_data = download_and_concat_Tdrive_files('99', 'T_Drive_files')

In [4]:
t_drive_42_data.to_csv(r'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Service Requests/2022/2022-81 XPEF persons vs mgra13_based_inputs/DS42_Aggregated.csv')

In [5]:
t_drive_99_data.to_csv(r'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Service Requests/2022/2022-81 XPEF persons vs mgra13_based_inputs/DS99_Aggregated.csv')

In [8]:
core_cols = [x for x in t_drive_42_data.columns if ('emp' in x)]
columns_of_interest = ['year', 'mgra'] + core_cols

In [9]:
columns_of_interest.remove('empden')
columns_of_interest.remove('retempden')
columns_of_interest.remove('empdenbin')

In [10]:
t_drive_42_data_emp = t_drive_42_data[columns_of_interest]
t_drive_42_data_emp = t_drive_42_data_emp.set_index(['year', 'mgra'])

t_drive_99_data_emp = t_drive_99_data[columns_of_interest]
t_drive_99_data_emp = t_drive_99_data_emp.set_index(['year', 'mgra'])

In [16]:
(t_drive_42_data_emp - t_drive_99_data_emp).to_csv(r'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Service Requests/2022/2022-81 XPEF persons vs mgra13_based_inputs/Data/DS42_Minus_DS99.csv')

In [49]:
no_emp_total = core_cols
print('emp_total' in no_emp_total)
no_emp_total.remove('emp_total')
print('emp_total' in no_emp_total)

True
False


In [55]:
output = pd.DataFrame(t_drive_42_data_emp[no_emp_total].sum(axis=1))
output['emp_total'] = t_drive_42_data_emp['emp_total']
output.columns = ['Summed_Emp_values', 'emp_total']
output['Diff'] = output['Summed_Emp_values'] - output['emp_total']

output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Service Requests\2022\2022-81 XPEF persons vs mgra13_based_inputs\Results\3_emp_sums_minues_emp_total_DS42.csv')

In [11]:
'emp' in 'employment'

True

In [9]:
count_2 = household_number_comparison_houseolds_and_input_files('42', to_jdrive=True)
count_2

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,house_count_household_file,hh_count_input_files,Diff
0,2016,1,18,18,0
1,2016,2,34,34,0
2,2016,3,52,52,0
3,2016,4,30,30,0
4,2016,5,28,28,0
...,...,...,...,...,...
236889,2050,22995,37,37,0
236890,2050,22996,103,103,0
236891,2050,22998,87,87,0
236892,2050,23000,126,126,0


In [8]:
comp_1 = population_comparison_households_and_input_files('42', gq_only=True, no_gq=False, to_jdrive=True)
comp_1

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,pop_input_files,Diff
0,2016,1065,193,193,0
1,2016,1070,178,178,0
2,2016,1071,31,31,0
3,2016,1073,49,49,0
4,2016,1076,588,588,0
...,...,...,...,...,...
579,2050,19191,2694,2694,0
580,2050,19214,566,566,0
581,2050,22593,577,577,0
582,2050,22705,1074,1074,0


In [2]:
test_1 = download_and_concat_Tdrive_files('DS42', 'Person_Files')
test_1

,hhid,perid,household_serial_no,pnum,age,sex,miltary,pemploy,pstudent,ptype,...,grade,occen5,occsoc5,indcen,weeks,hours,rac1p,hisp,version,year
0,1,1,0,1,72,2,0,3,3,5,...,0,0,00-0000,0,5,0,1,1,0,2016
1,1,2,0,2,75,1,0,1,3,1,...,0,0,11-1021,0,1,35,1,1,0,2016
2,2,3,0,1,85,2,0,3,3,5,...,0,0,00-0000,0,5,0,1,1,0,2016
3,2,4,0,2,67,1,0,1,3,1,...,0,0,11-1021,0,1,35,1,1,0,2016
4,3,5,0,1,10,2,0,4,1,7,...,2,0,00-0000,0,5,0,1,1,0,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3699367,1455751,3699368,0,1,19,1,0,3,2,3,...,6,0,00-0000,0,5,0,1,1,0,2050
3699368,1455752,3699369,0,1,19,2,0,3,2,3,...,6,0,00-0000,0,5,0,1,1,0,2050
3699369,1455753,3699370,0,1,18,1,0,3,2,3,...,6,0,00-0000,0,5,0,6,1,0,2050
3699370,1455754,3699371,0,1,18,2,0,2,2,3,...,6,0,41-1011,0,5,0,6,1,0,2050


In [ ]:
test_1['hhid']

In [ ]:
def population_from_persons_dataset(dsid, gq_only, no_gq):
    '''Downloads and aggregates population data from the households dataset depending on GQ preference.'''
    # From households file
    person_file_all = download_and_concat_Tdrive_files(
        dsid, 'Person_Files')
    person_file_subset = person_file_all[['year', 'hhid', 'perid']] #Last value here doesn't matter which we grab as we are taking the count of it 
    person_file_subset['year'] = pd.to_numeric(
        person_file_subset['year'])

    if gq_only:
        person_file_subset = person_file_subset[person_file_all['Military'] == 1]
    elif no_gq:
        person_file_subset = person_file_subset[person_file_all['Military'] == 0]

    population_person_file = person_file_subset.groupby(
        ['year', 'hhid']).count().reset_index()
    population_person_file.columns = [
        'year', 'hhid', 'pop_count_person_file']
    return population_household_file

In [3]:
test_2 = download_and_concat_Tdrive_files('DS42', 'Household_Files')
test_2

,hhid,household_serial_no,taz,mgra,hinccat1,hinc,hworkers,veh,persons,hht,bldgsz,unittype,version,poverty,year
0,1,0,3458,47,5,294757,1,0,2,1,2,0,0,22.364,2016
1,2,0,3407,10,5,210480,1,0,2,1,2,0,0,15.970,2016
2,3,0,3458,16,5,236389,0,0,2,3,2,0,0,15.728,2016
3,4,0,3379,27,4,121977,2,0,2,2,2,0,0,8.353,2016
4,5,0,3358,3,4,104512,2,0,2,7,2,0,0,7.157,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455750,1455751,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455751,1455752,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455752,1455753,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455753,1455754,0,3144,6055,1,14000,1,0,1,0,9,1,0,1.234,2050


In [10]:
new = population_from_households_dataset('DS42', gq_only=False, no_gq=False)
new

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
237160,2050,22995,99
237161,2050,22996,269
237162,2050,22998,221
237163,2050,23000,282


In [4]:
new = population_comparison_houseolds_and_input_files('46', gq_only=False, no_gq=True, to_jdrive=False)
new

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


['year', 'mgra', 'pop_count_household_file', 'hhp']
   year  mgra  pop_count_household_file  hhp
0  2016     1                        41   41
1  2016     2                        81   81
2  2016     3                       111  111
3  2016     4                        73   73
4  2016     5                        63   63
no_GQ


,year,mgra,pop_count_household_file,pop_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
237696,2050,22996,236,236,0
237697,2050,22998,196,196,0
237698,2050,23000,307,307,0
237699,2050,23001,184,184,0


In [12]:
output = population_from_households_dataset('46', gq_only=False, no_gq=True)
output 

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
237696,2050,22996,236
237697,2050,22998,196
237698,2050,23000,307
237699,2050,23001,184


In [13]:
output

,year,mgra,pop_count_household_file
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
237696,2050,22996,236
237697,2050,22998,196
237698,2050,23000,307
237699,2050,23001,184


In [14]:
mgra_data = pd.read_csv(rf'J:\DataScience\DataQuality\QAQC\forecast_automation\mgra_series_13_outputs_CSV_data\aggregated_data\mgra_DS46_ind_QA.csv', usecols=['year', 'mgra','hhp'])
mgra_data

,mgra,hhp,year
0,1,41,2016
1,2,81,2016
2,3,111,2016
3,4,73,2016
4,5,63,2016
...,...,...,...
299021,22998,196,2050
299022,22999,0,2050
299023,23000,307,2050
299024,23001,184,2050


In [19]:
df3 = mgra_data.merge(output, how='left', on=['year', 'mgra'])
df3 = df3[['year', 'mgra', 'hhp', 'pop_count_household_file']]
df3['Diff'] = df3['hhp'] - df3['pop_count_household_file']

df3.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2022\2022-73 DS39 (Revised) Forecast Output QC\results\mgra_population_comaprison_ds46_no_GQ.csv')

In [15]:
set(output['year'])


{2016, 2018, 2020, 2023, 2025, 2026, 2029, 2030, 2032, 2035, 2040, 2045, 2050}

In [16]:
set(mgra_data['year'])

{2016, 2018, 2020, 2023, 2025, 2026, 2029, 2030, 2032, 2035, 2040, 2045, 2050}

In [2]:
a = population_from_households_dataset('46', gq_only=False, no_gq=False)
b = population_from_households_dataset('46', gq_only=True, no_gq=False)
c = population_from_households_dataset('46', gq_only=False, no_gq=True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


In [9]:
def return_a_value(n):
    assert n > 5

    return 'Yay'

In [10]:
return_a_value(5)

AssertionError: 

In [6]:
a

,year,mgra,pop_count_household_file
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
237975,2050,22996,236
237976,2050,22998,196
237977,2050,23000,307
237978,2050,23001,184


In [5]:
b

,year,mgra,pop_count_household_file
0,2016,1065,193
1,2016,1070,178
2,2016,1071,31
3,2016,1073,49
4,2016,1076,588
...,...,...,...
579,2050,19191,2694
580,2050,19214,566
581,2050,22593,577
582,2050,22705,1074


In [4]:
c

,year,mgra,pop_count_household_file
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
237696,2050,22996,236
237697,2050,22998,196
237698,2050,23000,307
237699,2050,23001,184


In [7]:
980-701

279

In [3]:
d = population_from_households_dataset('46', gq_only=True, no_gq=True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


In [8]:
d

,year,mgra,pop_count_household_file
0,2016,1065,193
1,2016,1070,178
2,2016,1071,31
3,2016,1073,49
4,2016,1076,588
...,...,...,...
579,2050,19191,2694
580,2050,19214,566
581,2050,22593,577
582,2050,22705,1074


In [ ]:
def population_from_households_dataset(dsid, gq_only, no_gq):
    '''Downloads and aggregates population data from the households dataset depending on GQ preference.'''
    # From households file
    household_file_all = download_and_concat_Tdrive_files(dsid, 'Household_Files')
    household_file_subset = household_file_all[['year', 'mgra', 'persons']]
    household_file_subset['year'] = pd.to_numeric(household_file_subset['year'])

    if gq_only:
        household_file_subset = household_file_subset[household_file_all['unittype'] == 1]
    elif no_gq:
        household_file_subset = household_file_subset[household_file_all['unittype'] == 0]

    population_household_file = household_file_subset.groupby(['year', 'mgra']).sum().reset_index()
    population_household_file.columns = ['year', 'mgra', 'pop_count_household_file']
    return population_household_file

    if (gq_only == no_gq) & (gq_only == False):
        # GQ and Non-GQ Person Comparison
        population_household_file_all = household_file_subset.groupby(
            ['year', 'mgra']).sum().reset_index()
        population_household_file_all.columns = [
            'year', 'mgra', 'pop_count_household_file']
        return population_household_file_all
    elif gq_only == True:
        # Just GQ Population Comparison
        household_file_gq_only = household_file_subset[household_file_all['unittype'] == 1]
        population_household_file_gq_only = household_file_gq_only.groupby(
            ['year', 'mgra']).sum().reset_index()
        population_household_file_gq_only.columns = [
            'year', 'mgra', 'pop_count_household_file']
        return population_household_file_gq_only
    else:
        # Just Non-GQ Person Comparison
        household_file_no_gq = household_file_subset[household_file_all['unittype'] == 0]
        population_household_file_no_gq = household_file_no_gq.groupby(
            ['year', 'mgra']).sum().reset_index()
        population_household_file_no_gq.columns = [
            'year', 'mgra', 'pop_count_household_file']
        return population_household_file_no_gq

In [4]:
output = population_comparison_houseolds_and_input_files('46', gq_only=False, no_gq=True, to_jdrive=False)
output

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,pop_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
237696,2050,22996,236,236,0
237697,2050,22998,196,196,0
237698,2050,23000,307,307,0
237699,2050,23001,184,184,0


In [ ]:
def population_comparison_houseolds_and_input_files(dsid, gq_only, no_gq, to_jdrive):
    '''Compare MGRA population data to household dataset population data based on gq preference.'''
    # Input Files
    mgra_data = pd.read_csv(rf'J:\DataScience\DataQuality\QAQC\forecast_automation\mgra_series_13_outputs_CSV_data\aggregated_data\mgra_DS{dsid}_ind_QA.csv', usecols=[
                            'year', 'mgra', 'pop', 'hhp'])
    mgra_data['gq_pop_input_files'] = mgra_data['pop'] - mgra_data['hhp']
    mgra_data = mgra_data.rename(
        columns={'pop': 'pop_input_files', 'hhp': 'hhp_input_files'})

    # Concat and merge
    output = population_from_households_dataset(dsid, gq_only, no_gq).merge(mgra_data, how='left', on=['year', 'mgra'])
    output = output[['year', 'mgra', 'pop_household_file','pop_input_files']]
    output['Diff'] = output['pop_household_file'] - output['pop_input_files']

    if (gq_only == no_gq) & (gq_only == False):
        output_type = 'all'
    elif gq_only:
        output_type = 'GQ_only'
    else:
        output_type = 'no_GQ'

    if to_jdrive:
        output.to_csv(
            rf"J:\DataScience\DataQuality\QAQC\forecast_automation\mgra_series_13_outputs_CSV_data\other_outputs\mgra_households_dataset_population_comparison_{output_type}_DS{dsid}_QA.csv", index=False)

    return output

In [3]:
# a =population_comparison_houseolds_and_input_files('39', gq_only=False, no_gq=False, to_jdrive=True)
# print('A')
# b = population_comparison_houseolds_and_input_files('39', gq_only=True, no_gq=False, to_jdrive=True)
# print('B')
# c = population_comparison_houseolds_and_input_files('39', gq_only=False, no_gq=True, to_jdrive=True)
# print('c')

# d = population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=False, to_jdrive=True)
# print('d')
# e = population_comparison_houseolds_and_input_files('41', gq_only=True, no_gq=False, to_jdrive=True)
# print('e')
# f = population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=True, to_jdrive=True)
# print('f')

# g = population_comparison_houseolds_and_input_files('46', gq_only=False, no_gq=False, to_jdrive=True)
# print('g')
# h = population_comparison_houseolds_and_input_files('46', gq_only=True, no_gq=False, to_jdrive=True)
# print('h')
i = population_comparison_houseolds_and_input_files('46', gq_only=False, no_gq=True, to_jdrive=True)
print('i')

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


i


In [6]:
a = household_number_comparison_houseolds_and_input_files('39', True)
print('1')
b = household_number_comparison_houseolds_and_input_files('41', True)
print('2')
c = household_number_comparison_houseolds_and_input_files('46', True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


1
2


In [2]:
final = population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=False, to_jdrive=False)
final

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_input_files,pop_count_household_file,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
238574,2050,22996,237,237,0
238575,2050,22998,218,218,0
238576,2050,23000,312,312,0
238577,2050,23001,195,195,0


In [3]:
final = population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=False, to_jdrive=False)
final

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,pop_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
238574,2050,22996,237,237,0
238575,2050,22998,218,218,0
238576,2050,23000,312,312,0
238577,2050,23001,195,195,0


In [4]:
population_comparison_houseolds_and_input_files('39', gq_only=False, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('39', gq_only=True, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('39', gq_only=False, no_gq=True, to_jdrive=True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,hhp_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
237715,2050,22996,228,228,0
237716,2050,22998,184,184,0
237717,2050,23000,298,298,0
237718,2050,23001,194,194,0


In [5]:
population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('41', gq_only=True, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=True, to_jdrive=True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,hhp_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
238293,2050,22996,237,237,0
238294,2050,22998,218,218,0
238295,2050,23000,312,312,0
238296,2050,23001,195,195,0


In [6]:
population_comparison_houseolds_and_input_files('46', gq_only=False, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('46', gq_only=True, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('46', gq_only=False, no_gq=True, to_jdrive=True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,hhp_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
237696,2050,22996,236,236,0
237697,2050,22998,196,196,0
237698,2050,23000,307,307,0
237699,2050,23001,184,184,0
